# Proyecto final

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz

In [1]:
import pyomo.environ as pe
import pyomo.opt as po
from pyomo.environ import NonNegativeReals, Binary

#### Create the model 

In [2]:
model = pe.ConcreteModel()

#### Sets


In [3]:
# days
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

# periods
PERIODS = ["Morning", "Afternoon"]

# hours (time slots) within each period
HM = [1,2,3,4,5]  # Morning slots
HA = [1,2,3,4,5]  # Afternoon slots

ACTIVITIES = ["Yoga", "CommunityMeeting", "KidsClass", "CraftsWorkshop"]

SUPERVISORS = ["Supervisor1", "Supervisor2"]

model.d = pe.Set(initialize=DAYS)
model.p = pe.Set(initialize=PERIODS)
model.a = pe.Set(initialize=ACTIVITIES)
model.s = pe.Set(initialize=SUPERVISORS)

# hours of the morning and afternoon
model.Hm = pe.Set(initialize=HM)
model.Ha = pe.Set(initialize=HA)

def H_init(m, p):
    return HM if p == "Morning" else HA

# H[p] = hours available in that period
model.H = pe.Set(model.p, initialize=H_init)  

model.dph = pe.Set(
    dimen=3,
    initialize=[(d,p,h) for d in DAYS for p in PERIODS for h in (HM if p=="Morning" else HA)]
)


#### Parameters


In [4]:
# room availability room_avail[(d,p,h)] binary
#since this data is ficticious we have decided some random values for these parameters:
room_avail = {(d,p,h): 1 for d in DAYS for p in PERIODS for h in (HM if p=="Morning" else HA)}

# min weekly sessions per activity min_sessions[a]
#at least one session per activity 
min_sessions = {a: 1 for a in ACTIVITIES}

# max weekly sessions per activity max_sessions[a]
# maximum five sessions per activity 
max_sessions = {a: 5 for a in ACTIVITIES}

# duration in consecutive slots dur[a]
dur = {
    "Yoga": 2,
    "CommunityMeeting": 1,
    "KidsClass": 2,
    "CraftsWorkshop": 1
}

# needs supervisor needs_sup[a] in (0 1)
# all activities require supervisor 
needs_sup = {a: 1 for a in ACTIVITIES}

# supervisor availability sup_avail[(s,d,p,h)] in (0 1)
# supervisors unavailable sunday afternoon
sup_avail = {
    (s,d,p,h): 0 if (d=="Sunday" and p=="Afternoon") else 1
    for s in SUPERVISORS
    for d in DAYS
    for p in PERIODS
    for h in (HM if p=="Morning" else HA)
}

# room capacity scalar
CAPACITY = 25

# demand limit per session
demand = {
    (a,d,p,h): 15 if a=="Yoga"
    else 20 if a=="KidsClass"
    else 10
    for a in ACTIVITIES
    for d in DAYS
    for p in PERIODS
    for h in (HM if p=="Morning" else HA)
}


In [5]:
model.room_avail = pe.Param(
    model.dph,
    initialize=lambda m,d,p,h: room_avail[(d,p,h)],
    within=pe.Binary
)

model.min_sessions = pe.Param(
    model.a,
    initialize=lambda m,a: min_sessions[a],
    within=pe.NonNegativeIntegers
)

model.max_sessions = pe.Param(
    model.a,
    initialize=lambda m,a: max_sessions[a],
    within=pe.NonNegativeIntegers
)

model.dur = pe.Param(
    model.a,
    initialize=lambda m,a: dur[a],
    within=pe.PositiveIntegers
)

model.needs_sup = pe.Param(
    model.a,
    initialize=lambda m,a: needs_sup[a],
    within=pe.Binary
)

model.sup_avail = pe.Param(
    model.s, model.dph,
    initialize=lambda m,s,d,p,h: sup_avail[(s,d,p,h)],
    within=pe.Binary
)

model.capacity = pe.Param(
    initialize=CAPACITY,
    within=pe.PositiveIntegers
)

model.demand = pe.Param(
    model.a, model.dph,
    initialize=lambda m,a,d,p,h: demand[(a,d,p,h)],
    within=pe.NonNegativeIntegers
)


#### Variables


In [6]:
# binary (1 if activity a starts on day d at period p time h else 0)
model.X = pe.Var(model.a, model.dph, within=pe.Binary)

# number of attendees of activity a that starts on day d at period d at time slot h
model.n = pe.Var(model.a, model.dph, within=pe.NonNegativeIntegers)

# binary (1 if supervisor s is assigned to activity a at d,p,h)
model.y = pe.Var(model.a, model.dph, model.s, within=pe.Binary)

# auxiliary variable (minimum number of persons among all sessions)
model.Z = pe.Var(within=pe.NonNegativeIntegers)

# binary (1 if activity a is running on day d at period p time h else 0)
model.R = pe.Var(model.a, model.dph, within=pe.Binary)


#### Objective Function


In [7]:
def obj_rule(m):
    return m.Z

model.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)


#### Constraints

In [8]:
# valid start time slots
def valid_start(m, a, p, h):
    return h + pe.value(m.dur[a]) - 1 <= max(m.H[p])

In [9]:
# activity running definition
def running_link_rule(m, a, d, p, h):
    return m.R[a,(d,p,h)] == sum(
        m.X[a,(d,p,t)]
        for t in m.H[p]
        if valid_start(m,a,p,t) and t <= h <= t + pe.value(m.dur[a]) - 1
    )

model.running_link = pe.Constraint(model.a, model.dph, rule=running_link_rule)


In [10]:
# invalid starts forced to zero
def invalid_start_rule(m, a, d, p, h):
    if valid_start(m,a,p,h):
        return pe.Constraint.Skip
    return m.X[a,(d,p,h)] == 0

model.invalid_start = pe.Constraint(model.a, model.dph, rule=invalid_start_rule)


In [11]:
# room availability
def room_availability_rule(m, d, p, h):
    return sum(m.R[a,(d,p,h)] for a in m.a) <= m.room_avail[d,p,h]

model.room_availability = pe.Constraint(model.dph, rule=room_availability_rule)


In [12]:

# no overlapping activities
def no_overlap_rule(m, d, p, h):
    return sum(m.R[a,(d,p,h)] for a in m.a) <= 1

model.no_overlap = pe.Constraint(model.dph, rule=no_overlap_rule)


In [13]:
# supervisor required
def supervisor_required_rule(m, a, d, p, h):
    return sum(m.y[a,(d,p,h),s] for s in m.s) >= m.needs_sup[a] * m.X[a,(d,p,h)]

model.supervisor_required = pe.Constraint(model.a, model.dph, rule=supervisor_required_rule)


In [14]:
# supervisor availability for full duration
model.sup_avail_full = pe.ConstraintList()

for a in model.a:
    for (d,p,h) in model.dph:
        if not valid_start(model,a,p,h):
            continue
        dur_a = int(pe.value(model.dur[a]))
        for s in model.s:
            for k in range(h, h + dur_a):
                model.sup_avail_full.add(
                    model.y[a,(d,p,h),s] <= model.sup_avail[s,d,p,k]
                )


In [15]:
# supervisor no overlap
def supervisor_no_overlap_rule(m, s, d, p, h):
    return sum(
        m.y[a,(d,p,t),s]
        for a in m.a
        for t in m.H[p]
        if valid_start(m,a,p,t) and t <= h <= t + pe.value(m.dur[a]) - 1
    ) <= 1

model.supervisor_no_overlap = pe.Constraint(model.s, model.dph, rule=supervisor_no_overlap_rule)


In [16]:
# capacity constraint
def capacity_rule(m, a, d, p, h):
    return m.n[a,(d,p,h)] <= m.capacity * m.X[a,(d,p,h)]

model.capacity_constraint = pe.Constraint(model.a, model.dph, rule=capacity_rule)


In [17]:
# demand constraint
def demand_rule(m, a, d, p, h):
    return m.n[a,(d,p,h)] <= m.demand[a,d,p,h] * m.X[a,(d,p,h)]

model.demand_constraint = pe.Constraint(model.a, model.dph, rule=demand_rule)


In [18]:
# minimum weekly sessions per activity
def min_sessions_rule(m, a):
    return sum(
        m.X[a,(d,p,h)]
        for (d,p,h) in m.dph
        if valid_start(m,a,p,h)
    ) >= m.min_sessions[a]

model.min_sessions_constraint = pe.Constraint(model.a, rule=min_sessions_rule)


In [19]:
# maximum weekly sessions per activity
def max_sessions_rule(m, a):
    return sum(
        m.X[a,(d,p,h)]
        for (d,p,h) in m.dph
        if valid_start(m,a,p,h)
    ) <= m.max_sessions[a]

model.max_sessions_constraint = pe.Constraint(model.a, rule=max_sessions_rule)


In [20]:
# z definition
M = CAPACITY

def z_rule(m, a, d, p, h):
    return m.Z <= m.n[a,(d,p,h)] + M * (1 - m.X[a,(d,p,h)])

model.z_constraint = pe.Constraint(model.a, model.dph, rule=z_rule)


#### Solve model

In [21]:
solver = pe.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Mostrar estado de la optimización
print("Solver Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2707518
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\Guill\AppData\Local\Temp\tmpysn2q08_.pyomo.lp
Reading time = 0.08 seconds
x1: 2444 rows, 1401 columns, 5992 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2444 rows, 1401 columns and 5992 nonzeros
Model fingerprint: 0x3b64c3cc
Variable types: 0 continuous, 1401 integer (1120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 1942 rows and 698 columns
Presolve time: 0.05s
Presolved: 502 rows, 703 columns, 2483 nonzero

In [22]:
print("solution")
for (d,p,h) in model.dph:
    for a in model.a:
        if pe.value(model.X[a,(d,p,h)]) > 0.5:
            sups = [s for s in model.s if pe.value(model.y[a,(d,p,h),s]) > 0.5]
            print(d,p,h,a,"attendees",int(pe.value(model.n[a,(d,p,h)])),"supervisors",sups)

print("z value",int(pe.value(model.Z)))


solution
Tuesday Afternoon 1 Yoga attendees 15 supervisors ['Supervisor1']
Wednesday Morning 4 KidsClass attendees 20 supervisors ['Supervisor1']
Friday Afternoon 3 CraftsWorkshop attendees 10 supervisors ['Supervisor1', 'Supervisor2']
Sunday Morning 4 CommunityMeeting attendees 10 supervisors ['Supervisor2']
z value 10
